# Disease importance analysis

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import glob
root = "../"

In [2]:
dict_uf_cod = {11: 'RO', 12: 'AC', 13: 'AM', 14: 'RR', 15: 'PA', 16: 'AP', 17: 'TO',
21: 'MA', 22: 'PI', 23: 'CE', 24: 'RN', 25: 'PB', 26: 'PE', 27: 'AL', 28: 'SE',
29: 'BA', 31: 'MG', 32: 'ES', 33: 'RJ', 35: 'SP', 41: 'PR', 42: 'SC', 43: 'RS',
50: 'MS', 51: 'MT', 52: 'GO', 53: 'DF'}
to_remove = list(dict_uf_cod.values()) + ["PREVIOUS"]

In [3]:
spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
spearman.head(15)

,Doenças,Correlação com suicidio,P value,Qtd Municipios
0,TRANSTORNOS_DE_HUMOR_[AFETIVOS],0.422815,6.807913e-221,5113
1,TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁL...,0.412652,8.880540e-209,5096
2,NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES,0.407627,4.439103e-201,5040
3,ARTROSE,0.403773,5.851281e-187,4781
4,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,0.391823,3.067493e-181,4947
5,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,0.391497,1.579413e-195,5351
6,COLELITÍASE_E_COLECISTITE,0.381351,7.796045e-186,5381
7,TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT,0.380755,1.343290e-165,4805
8,NEOPLASIA_MALIGNA_DA_PELE,0.376219,1.487533e-144,4297
9,OUTRAS_DOENÇAS_ISQUÊMICAS_DO_CORAÇÃO,0.372285,4.552244e-176,5367


In [4]:
bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
bivariate.head(15)

,Doenças,Moran BV I
0,ARTROSE,0.370397
1,NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES,0.353406
2,TRANSTORNOS_DE_HUMOR_[AFETIVOS],0.349508
3,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,0.318787
4,DOENÇAS_DO_APÊNDICE,0.306851
5,NEOPL_MALIG_OUTR_LOCALIZ_MAL_DEF_SECUN_E_NÃO_E...,0.303096
6,COLELITÍASE_E_COLECISTITE,0.297942
7,TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁL...,0.296933
8,EMBOLIA_E_TROMBOSE_ARTERIAIS,0.294615
9,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,0.287545


# Todos arquivos
- Valor absoluto das importâncias
- Normalização com MinMaxScaler

In [5]:
all_files = glob.glob(root + "CSV/Importance/*")
all_files = [x.split("\\")[1] for x in all_files]
all_files.remove("bivariate_importance.csv")
all_files.remove('spearman_importance.csv')

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
x = np.abs(bivariate[["Moran BV I"]].values)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
bivariate["Bivariate"] = x_scaled
final_df = bivariate[["Doenças", "Bivariate"]]

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
x = np.abs(spearman[["Correlação com suicidio"]].values)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
spearman["Spearman"] = x_scaled
spearman = spearman[["Doenças", "Spearman"]]
final_df = pd.merge(final_df, spearman, how='outer')

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.rename(columns={"Feature": "Doenças", "Avg_importance": file})
    df = df[~df['Doenças'].isin(to_remove)]
    x = np.abs(df[[file]].values)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df[file] = x_scaled
    final_df = pd.merge(final_df, df, how='outer')
    
final_df = final_df.fillna(0)
final_df["Importance"] = final_df.drop(columns="Doenças").sum(axis=1)
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(30).reset_index(drop=True)

,Doenças,Importance
0,COLELITÍASE_E_COLECISTITE,11.398649
1,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,11.011713
2,DOENÇAS_DO_APÊNDICE,9.912131
3,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,9.573663
4,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,9.479954
5,OUTRAS_HÉRNIAS,9.230332
6,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,8.193774
7,NEOPLASIA_MALIGNA_DO_CÓLON,7.998903
8,NEOPLASIA_MALIGNA_DO_PÂNCREAS,7.563167
9,PARTO_ÚNICO_ESPONTÂNEO,7.344360


## Somente Random Forest

In [6]:
all_files = ["fi_random_forest_highest_rates.csv",
"fi_random_forest_regression.csv",
"fi_random_forest_satscan.csv", 
"fi_shap_random_forest.csv", 
"fi_shap_random_forest_highest_rates.csv",
"fi_shap_random_forest_satscan.csv"
]

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
x = np.abs(bivariate[["Moran BV I"]].values)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
bivariate["Bivariate"] = x_scaled
final_df = bivariate[["Doenças", "Bivariate"]]

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
x = np.abs(spearman[["Correlação com suicidio"]].values)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
spearman["Spearman"] = x_scaled
spearman = spearman[["Doenças", "Spearman"]]
final_df = pd.merge(final_df, spearman, how='outer')

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.rename(columns={"Feature": "Doenças", "Avg_importance": file})
    df = df[~df['Doenças'].isin(to_remove)]
    x = np.abs(df[[file]].values)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df[file] = x_scaled
    final_df = pd.merge(final_df, df, how='outer')
    
final_df = final_df.fillna(0)
final_df["Importance"] = final_df.drop(columns="Doenças").sum(axis=1)
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(30).reset_index(drop=True)

,Doenças,Importance
0,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,6.669501
1,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,4.051359
2,NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES,3.652204
3,DOENÇAS_DO_APÊNDICE,3.337970
4,COLELITÍASE_E_COLECISTITE,3.169403
5,TRANSTORNOS_DE_HUMOR_[AFETIVOS],2.996363
6,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,2.901698
7,TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT,2.887722
8,NEOPL_MALIG_OUTR_LOCALIZ_MAL_DEF_SECUN_E_NÃO_E...,2.626907
9,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,2.493843


## Somente SHAP

In [7]:
all_files = glob.glob(root + "CSV/Importance/fi_shap*")
all_files = [x.split("\\")[1] for x in all_files]

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
x = np.abs(bivariate[["Moran BV I"]].values)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
bivariate["Bivariate"] = x_scaled
final_df = bivariate[["Doenças", "Bivariate"]]

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
x = np.abs(spearman[["Correlação com suicidio"]].values)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
spearman["Spearman"] = x_scaled
spearman = spearman[["Doenças", "Spearman"]]
final_df = pd.merge(final_df, spearman, how='outer')

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.rename(columns={"Feature": "Doenças", "Avg_importance": file})
    df = df[~df['Doenças'].isin(to_remove)]
    x = np.abs(df[[file]].values)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df[file] = x_scaled
    final_df = pd.merge(final_df, df, how='outer')
    
final_df = final_df.fillna(0)
final_df["Importance"] = final_df.drop(columns="Doenças").sum(axis=1)
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(30).reset_index(drop=True)

,Doenças,Importance
0,COLELITÍASE_E_COLECISTITE,7.254977
1,DOENÇAS_DO_APÊNDICE,6.671440
2,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,5.889620
3,OUTRAS_HÉRNIAS,5.586167
4,NEOPLASIA_MALIGNA_DO_CÓLON,5.243770
5,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,5.203600
6,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,5.149863
7,PARTO_ÚNICO_ESPONTÂNEO,5.116724
8,NEOPL_MALIG_JUNÇÃO_RETOSSIGM_RETO_ÂNUS_CANAL_ANAL,4.739155
9,NEOPLASIA_MALIGNA_DA_MAMA,4.722635


## Tamanho reduzido
- Limitando os 100 primeiros
- Sem np.abs

In [8]:
all_files = glob.glob(root + "CSV/Importance/*")
all_files = [x.split("\\")[1] for x in all_files]
for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    print(str(file) + ": " + str(df.shape[0]))

bivariate_importance.csv: 313
fi_coef_elastic_net.csv: 111
fi_coef_linear_regression.csv: 111
fi_coef_linear_svc_highest_rates.csv: 340
fi_coef_linear_svc_satscan.csv: 340
fi_coef_logistic_regression_highest_rates.csv: 340
fi_coef_logistic_regression_satscan.csv: 340
fi_random_forest_highest_rates.csv: 179
fi_random_forest_regression.csv: 111
fi_random_forest_satscan.csv: 117
fi_shap_elastic_net.csv: 111
fi_shap_linear_regression.csv: 111
fi_shap_linear_svc_highest_rates.csv: 340
fi_shap_linear_svc_satscan.csv: 340
fi_shap_logistic_regression_highest_rates.csv: 340
fi_shap_logistic_regression_satscan.csv: 340
fi_shap_random_forest.csv: 111
fi_shap_random_forest_highest_rates.csv: 179
fi_shap_random_forest_satscan.csv: 117
spearman_importance.csv: 215


In [9]:
all_files = glob.glob(root + "CSV/Importance/*")
all_files = [x.split("\\")[1] for x in all_files]
all_files.remove("bivariate_importance.csv")
all_files.remove('spearman_importance.csv')

final_df = pd.DataFrame({})
for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.sort_values(by="Avg_importance", ascending=False).head(100)
    df = df.rename(columns={"Feature": "Doenças", "Avg_importance": file})
    df = df[~df['Doenças'].isin(to_remove)]
    x = df[[file]].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df[file] = x_scaled
    if final_df.shape[0] == 0:
        final_df = df
    else:
        final_df = pd.merge(final_df, df, how='outer')
    
final_df = final_df.fillna(0)
final_df["Importance"] = final_df.drop(columns=["Doenças"]).sum(axis=1)
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(30).reset_index(drop=True)

,Doenças,Importance
0,COLELITÍASE_E_COLECISTITE,9.758520
1,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,9.368446
2,DOENÇAS_DO_APÊNDICE,7.794030
3,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,7.325940
4,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,7.141125
5,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,6.221214
6,NEOPLASIA_MALIGNA_DO_CÓLON,5.848878
7,HEMORRÓIDAS,5.785533
8,NEOPLASIA_MALIGNA_DO_PÂNCREAS,5.622333
9,OUTRAS_HÉRNIAS,5.423159


## Somente melhores

In [10]:
all_files = ["fi_shap_random_forest.csv", "fi_shap_logistic_regression_highest_rates.csv", "fi_shap_linear_svc_satscan.csv",
"fi_coef_linear_svc_satscan.csv", "fi_random_forest_regression.csv", "fi_coef_logistic_regression_highest_rates.csv"]

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
x = np.abs(bivariate[["Moran BV I"]].values)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
bivariate["Bivariate"] = x_scaled
final_df = bivariate[["Doenças", "Bivariate"]]

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
x = np.abs(spearman[["Correlação com suicidio"]].values)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
spearman["Spearman"] = x_scaled
spearman = spearman[["Doenças", "Spearman"]]
final_df = pd.merge(final_df, spearman, how='outer')

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.rename(columns={"Feature": "Doenças", "Avg_importance": file})
    df = df[~df['Doenças'].isin(to_remove)]
    x = np.abs(df[[file]].values)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df[file] = x_scaled
    final_df = pd.merge(final_df, df, how='outer')
    
final_df = final_df.fillna(0)
final_df["Importance"] = final_df.drop(columns="Doenças").sum(axis=1)
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(30).reset_index(drop=True)

,Doenças,Importance
0,COLELITÍASE_E_COLECISTITE,5.819557
1,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,5.245429
2,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,5.116689
3,DOENÇAS_DO_APÊNDICE,4.989649
4,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,4.902126
5,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,4.766684
6,TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT,4.736582
7,NEOPLASIA_MALIGNA_DO_CÓLON,4.178638
8,NEOPLASIA_MALIGNA_DO_PÂNCREAS,4.079716
9,NEOPLASIA_MALIGNA_DO_ESÔFAGO,3.918251


In [11]:
all_files = ["fi_shap_random_forest.csv", "fi_shap_logistic_regression_highest_rates.csv", "fi_shap_linear_svc_satscan.csv",
"fi_coef_linear_svc_satscan.csv", "fi_random_forest_regression.csv", "fi_coef_logistic_regression_highest_rates.csv"]

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
x = bivariate[["Moran BV I"]].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
bivariate["Bivariate"] = x_scaled
final_df = bivariate[["Doenças", "Bivariate"]]

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
x = spearman[["Correlação com suicidio"]].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
spearman["Spearman"] = x_scaled
spearman = spearman[["Doenças", "Spearman"]]
final_df = pd.merge(final_df, spearman, how='outer')

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.rename(columns={"Feature": "Doenças", "Avg_importance": file})
    df = df[~df['Doenças'].isin(to_remove)]
    x = df[[file]].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df[file] = x_scaled
    final_df = pd.merge(final_df, df, how='outer')
    
final_df = final_df.fillna(0)
final_df["Importance"] = final_df.drop(columns="Doenças").sum(axis=1)
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(30).reset_index(drop=True)

,Doenças,Importance
0,COLELITÍASE_E_COLECISTITE,6.362739
1,DOENÇAS_DO_APÊNDICE,5.805421
2,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,5.752032
3,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,5.659195
4,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,5.607955
5,TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT,5.453598
6,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,5.282293
7,NEOPLASIA_MALIGNA_DO_CÓLON,4.941449
8,NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES,4.761998
9,NEOPLASIA_MALIGNA_DO_PÂNCREAS,4.738569


# Todos arquivos por frequência

In [12]:
import itertools
from collections import Counter

all_files = glob.glob(root + "CSV/Importance/*")
all_files = [x.split("\\")[1] for x in all_files]
all_files.remove("bivariate_importance.csv")
all_files.remove('spearman_importance.csv')

final_list = []

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
bivariate["Moran BV I"] = np.abs(bivariate["Moran BV I"])
bivariate = list(bivariate.sort_values(by="Moran BV I", ascending=False).head(30)["Doenças"])
final_list.append(bivariate)

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
spearman["Correlação com suicidio"] = np.abs(spearman["Correlação com suicidio"])
spearman = list(spearman.sort_values(by="Correlação com suicidio", ascending=False).head(30)["Doenças"])
final_list.append(spearman)

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df["Avg_importance"] = np.abs(df["Avg_importance"])
    df = df[~df['Feature'].isin(to_remove)]
    df = list(df.sort_values(by="Avg_importance", ascending=False).head(30)["Feature"])
    final_list.append(df)
    
flattened_list = list(itertools.chain.from_iterable(final_list))
Counter(flattened_list).most_common(20)

[('OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE', 19),
 ('BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC', 15),
 ('COLELITÍASE_E_COLECISTITE', 15),
 ('NEOPLASIA_MALIGNA_DO_CÓLON', 14),
 ('OUTRAS_HÉRNIAS', 14),
 ('DOENÇAS_DO_APÊNDICE', 13),
 ('TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTERVERT', 13),
 ('FRATURA_DO_FÊMUR', 13),
 ('NEOPL_MALIG_OUTR_LOCALIZ_MAL_DEF_SECUN_E_NÃO_ESPEC', 12),
 ('EMBOLIA_PULMONAR', 12),
 ('PARTO_ÚNICO_ESPONTÂNEO', 11),
 ('TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT', 11),
 ('OUTRAS_DOENÇAS_ISQUÊMICAS_DO_CORAÇÃO', 11),
 ('NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES', 10),
 ('NEOPLASIA_MALIGNA_DO_PÂNCREAS', 10),
 ('LUXAÇÕES_ENTORSE_DISTENSÃO_REG_ESP_E_MÚLTIP_CORPO', 10),
 ('TRANSTORNOS_DE_HUMOR_[AFETIVOS]', 9),
 ('NEOPL_MALIG_JUNÇÃO_RETOSSIGM_RETO_ÂNUS_CANAL_ANAL', 9),
 ('NEOPLASIA_MALIGNA_DO_ESÔFAGO', 8),
 ('HEMORRÓIDAS', 8)]

In [13]:
import itertools
from collections import Counter

# all_files = glob.glob(root + "CSV/Importance/*")
# all_files = [x.split("\\")[1] for x in all_files]
# all_files.remove("bivariate_importance.csv")
# all_files.remove('spearman_importance.csv')
all_files = ["fi_shap_random_forest.csv", "fi_shap_logistic_regression_highest_rates.csv", "fi_shap_linear_svc_satscan.csv"]

final_list = []

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
bivariate["Moran BV I"] = np.abs(bivariate["Moran BV I"])
bivariate = list(bivariate.sort_values(by="Moran BV I", ascending=False).head(10)["Doenças"])
final_list.append(bivariate)

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
spearman["Correlação com suicidio"] = np.abs(spearman["Correlação com suicidio"])
spearman = list(spearman.sort_values(by="Correlação com suicidio", ascending=False).head(10)["Doenças"])
final_list.append(spearman)

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df["Avg_importance"] = np.abs(df["Avg_importance"])
    df = df[~df['Feature'].isin(to_remove)]
    df = list(df.sort_values(by="Avg_importance", ascending=False).head(10)["Feature"])
    final_list.append(df)
    
flattened_list = list(itertools.chain.from_iterable(final_list))
Counter(flattened_list).most_common(20)

[('COLELITÍASE_E_COLECISTITE', 5),
 ('NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES', 3),
 ('TRANSTORNOS_DE_HUMOR_[AFETIVOS]', 3),
 ('BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC', 3),
 ('DOENÇAS_DO_APÊNDICE', 3),
 ('TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTERVERT', 3),
 ('TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT', 3),
 ('ARTROSE', 2),
 ('NEOPL_MALIG_OUTR_LOCALIZ_MAL_DEF_SECUN_E_NÃO_ESPEC', 2),
 ('TRANST_MENTAIS_E_COMPORTAMENTAIS_DEV_USO_DE_ÁLCOOL', 2),
 ('TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS', 2),
 ('OUTRAS_DOENÇAS_ISQUÊMICAS_DO_CORAÇÃO', 2),
 ('OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE', 2),
 ('OUTRAS_HÉRNIAS', 2),
 ('EMBOLIA_E_TROMBOSE_ARTERIAIS', 1),
 ('NEOPLASIA_MALIGNA_DA_PELE', 1),
 ('SÍFILIS_CONGÊNITA', 1),
 ('HEMORRÓIDAS', 1),
 ('ACID_VASCULAR_CEREBR_ISQUÊM_TRANSIT_E_SÍNDR_CORREL', 1),
 ('ABORTO_ESPONTÂNEO', 1)]

# Ponderação por posição

In [14]:
n=100

all_files = glob.glob(root + "CSV/Importance/*")
all_files = [x.split("\\")[1] for x in all_files]
all_files.remove("bivariate_importance.csv")
all_files.remove('spearman_importance.csv')

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
bivariate["Moran BV I"] = np.abs(bivariate["Moran BV I"])
bivariate = bivariate.sort_values(by="Moran BV I", ascending=False)[0:n]
bivariate["Bivariate"] = list(reversed(range(1,n+1)))
final_df = bivariate[["Doenças", "Bivariate"]]
final_df.head()

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
spearman["Correlação com suicidio"] = np.abs(spearman["Correlação com suicidio"])
spearman = spearman.sort_values(by="Correlação com suicidio", ascending=False)[0:n]
spearman["Spearman"] = list(reversed(range(1,n+1)))
spearman = spearman[["Doenças", "Spearman"]]
final_df = pd.merge(final_df, spearman, how='outer')

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.rename(columns={"Feature": "Doenças"})
    df = df[~df['Doenças'].isin(to_remove)]
    df["Avg_importance"] = np.abs(df["Avg_importance"])
    df = df.sort_values(by="Avg_importance", ascending=False)[0:n]
    df[file] = list(reversed(range(1,n+1)))
    df = df[["Doenças", file]]
    final_df = pd.merge(final_df, df, how='outer')
    
final_df = final_df.fillna(0)
final_df["Importance"] = final_df.drop(columns="Doenças").sum(axis=1)
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(10).reset_index(drop=True)

,Doenças,Importance
0,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,1786.0
1,COLELITÍASE_E_COLECISTITE,1648.0
2,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,1493.0
3,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,1461.0
4,OUTRAS_HÉRNIAS,1460.0
5,NEOPLASIA_MALIGNA_DO_CÓLON,1450.0
6,DOENÇAS_DO_APÊNDICE,1419.0
7,NEOPLASIA_MALIGNA_DO_PÂNCREAS,1352.0
8,TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT,1279.0
9,EMBOLIA_PULMONAR,1276.0


In [15]:
n=30

all_files = ["fi_shap_random_forest.csv", "fi_shap_logistic_regression_highest_rates.csv", "fi_shap_linear_svc_satscan.csv",
"fi_coef_linear_svc_satscan.csv", "fi_random_forest_regression.csv", "fi_coef_logistic_regression_highest_rates.csv"]

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
bivariate["Moran BV I"] = np.abs(bivariate["Moran BV I"])
bivariate = bivariate.sort_values(by="Moran BV I", ascending=False)[0:n]
bivariate["Bivariate"] = list(reversed(range(1,n+1)))
final_df = bivariate[["Doenças", "Bivariate"]]
final_df.head()

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
spearman["Correlação com suicidio"] = np.abs(spearman["Correlação com suicidio"])
spearman = spearman.sort_values(by="Correlação com suicidio", ascending=False)[0:n]
spearman["Spearman"] = list(reversed(range(1,n+1)))
spearman = spearman[["Doenças", "Spearman"]]
final_df = pd.merge(final_df, spearman, how='outer')

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.rename(columns={"Feature": "Doenças"})
    df = df[~df['Doenças'].isin(to_remove)]
    df["Avg_importance"] = np.abs(df["Avg_importance"])
    df = df.sort_values(by="Avg_importance", ascending=False)[0:n]
    df[file] = list(reversed(range(1,n+1)))
    df = df[["Doenças", file]]
    final_df = pd.merge(final_df, df, how='outer')
    
final_df = final_df.fillna(0)
final_df["Importance"] = final_df.drop(columns="Doenças").sum(axis=1)
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(n).reset_index(drop=True)

,Doenças,Importance
0,COLELITÍASE_E_COLECISTITE,177.0
1,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,152.0
2,DOENÇAS_DO_APÊNDICE,138.0
3,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,133.0
4,TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT,126.0
5,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,123.0
6,NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES,113.0
7,TRANSTORNOS_DE_HUMOR_[AFETIVOS],106.0
8,OUTRAS_HÉRNIAS,100.0
9,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,98.0


# Sem abs

In [16]:
all_files = glob.glob(root + "CSV/Importance/*")
all_files = [x.split("\\")[1] for x in all_files]
all_files.remove("bivariate_importance.csv")
all_files.remove('spearman_importance.csv')

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
x = bivariate[["Moran BV I"]].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
bivariate["Bivariate"] = x_scaled
final_df = bivariate[["Doenças", "Bivariate"]]

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
x = spearman[["Correlação com suicidio"]].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
spearman["Spearman"] = x_scaled
spearman = spearman[["Doenças", "Spearman"]]
final_df = pd.merge(final_df, spearman, how='outer')

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.rename(columns={"Feature": "Doenças", "Avg_importance": file})
    df = df[~df['Doenças'].isin(to_remove)]
    x = df[[file]].values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df[file] = x_scaled
    final_df = pd.merge(final_df, df, how='outer')
    
final_df = final_df.fillna(0)
final_df["Importance"] = final_df.drop(columns="Doenças").sum(axis=1)
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(30).reset_index(drop=True)

,Doenças,Importance
0,COLELITÍASE_E_COLECISTITE,13.086710
1,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,12.215934
2,DOENÇAS_DO_APÊNDICE,12.059122
3,TRANSTORNOS_DE_CONDUÇÃO_E_ARRITMIAS_CARDÍACAS,12.040048
4,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,11.416661
5,NEOPLASIA_MALIGNA_DO_CÓLON,10.095809
6,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,9.841491
7,HEMORRÓIDAS,9.548466
8,NEOPLASIA_MALIGNA_DO_PÂNCREAS,9.532696
9,NEOPLASIA_MALIGNA_DE_TRAQUÉIA_BRÔNQUIOS_E_PULMÕES,9.507243


In [17]:
all_files = glob.glob(root + "CSV/Importance/*")
all_files = [x.split("\\")[1] for x in all_files]

In [18]:
all_files

['bivariate_importance.csv',
 'fi_coef_elastic_net.csv',
 'fi_coef_linear_regression.csv',
 'fi_coef_linear_svc_highest_rates.csv',
 'fi_coef_linear_svc_satscan.csv',
 'fi_coef_logistic_regression_highest_rates.csv',
 'fi_coef_logistic_regression_satscan.csv',
 'fi_random_forest_highest_rates.csv',
 'fi_random_forest_regression.csv',
 'fi_random_forest_satscan.csv',
 'fi_shap_elastic_net.csv',
 'fi_shap_linear_regression.csv',
 'fi_shap_linear_svc_highest_rates.csv',
 'fi_shap_linear_svc_satscan.csv',
 'fi_shap_logistic_regression_highest_rates.csv',
 'fi_shap_logistic_regression_satscan.csv',
 'fi_shap_random_forest.csv',
 'fi_shap_random_forest_highest_rates.csv',
 'fi_shap_random_forest_satscan.csv',
 'spearman_importance.csv']

# Ponderação por posição (usando uma proporção)

Detectamos que os coefs são menos confiaveis do que os feature_importances e o SHAP.

Vamos testar uma proporção 0.2 * coefs + 0.4 * importance_RF + 0.4 * shap

In [19]:
n=100

all_files = glob.glob(root + "CSV/Importance/*")
all_files = [x.split("\\")[1] for x in all_files]
all_files.remove("bivariate_importance.csv")
all_files.remove('spearman_importance.csv')

bivariate = pd.read_csv(root + "CSV/Importance/bivariate_importance.csv", index_col=0)
bivariate["Moran BV I"] = bivariate["Moran BV I"]
bivariate = bivariate.sort_values(by="Moran BV I", ascending=False)[0:n]
bivariate["Bivariate"] = list(reversed(range(1,n+1)))
final_df = bivariate[["Doenças", "Bivariate"]]
final_df.head()

spearman = pd.read_csv(root + "CSV/Importance/spearman_importance.csv", index_col=0)
spearman["Correlação com suicidio"] = spearman["Correlação com suicidio"]
spearman = spearman.sort_values(by="Correlação com suicidio", ascending=False)[0:n]
spearman["Spearman"] = list(reversed(range(1,n+1)))
spearman = spearman[["Doenças", "Spearman"]]
final_df = pd.merge(final_df, spearman, how='outer')

for file in all_files:
    df = pd.read_csv(root + "CSV/Importance/" + file, index_col=0)
    df = df.rename(columns={"Feature": "Doenças"})
    df = df[~df['Doenças'].isin(to_remove)]
    df["Avg_importance"] = df["Avg_importance"]
    df = df.sort_values(by="Avg_importance", ascending=False)[0:n]
    df[file] = list(reversed(range(1,n+1)))
    df = df[["Doenças", file]]
    final_df = pd.merge(final_df, df, how='outer')

final_df = final_df.fillna(0)

# separando as colunas coef, shap e rf
coefs = [x for x in final_df.columns if 'fi_coef_' in x]
shaps = [x for x in final_df.columns if 'fi_shap_' in x]
rf_importances = list(set(final_df.columns) - set(coefs) - set(shaps))

# somando as colunas colunas coef, shap e rf
final_df['coefs_sum'] = final_df[coefs].sum(axis=1)
final_df['shaps_sum'] = final_df[shaps].sum(axis=1)
final_df['rf_importances_sum'] = final_df[rf_importances].sum(axis=1)

# importance = soma ponderada 0.2 * coef + 0.4 * shap + 0.4 * rf_importance
final_df["Importance"] = final_df['coefs_sum'] * 0.2 + final_df['shaps_sum'] * 0.4 + final_df['rf_importances_sum'] * 0.4
final_df.sort_values(by="Importance", ascending=False)[["Doenças", "Importance"]].head(30).reset_index(drop=True)

,Doenças,Importance
0,OUTRAS_NEOPLASIAS_MALIGNAS_DA_PELE,603.2
1,COLELITÍASE_E_COLECISTITE,577.8
2,BRONQUITE_ENFISEMA_E_OUTR_DOENÇ_PULM_OBSTR_CRÔNIC,549.2
3,NEOPLASIA_MALIGNA_DO_CÓLON,548.6
4,DOENÇAS_DO_APÊNDICE,545.6
5,TRANST_DISCAIS_CERVIC_E_OUTR_TRANST_DISC_INTER...,516.4
6,TRANST_MENTAIS_COMPORT_DEV_USO_OUTR_SUBST_PSICOAT,497.4
7,NEOPLASIA_MALIGNA_DO_PÂNCREAS,471.4
8,EMBOLIA_PULMONAR,466.4
9,NEOPL_MALIG_JUNÇÃO_RETOSSIGM_RETO_ÂNUS_CANAL_ANAL,466.0
